## Importações

In [ ]:
# Importação das bibliotecas necessárias
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D


## Preparação e normalização dos dados e geração dos rótulos

In [ ]:
# Pastas do dataset
dataset_dir = '/content/drive/MyDrive/Entregavel_2/imagens'
drone_dir = os.path.join(dataset_dir, '/content/drive/MyDrive/Entregavel_2/imagens/drone')
trator_dir = os.path.join(dataset_dir, '/content/drive/MyDrive/Entregavel_2/imagens/trator')

# Função para carregar imagens e gerar rótulols
def carregar_dataset(drone_dir, trator_dir):
    imagens = []
    rotulos = []

    # Carregar imagens de drones (classe 0)
    for img_name in os.listdir(drone_dir):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(drone_dir, img_name)
            img = load_img(img_path, target_size=(150, 150))
            img_array = img_to_array(img) / 255.0
            imagens.append(img_array)
            rotulos.append(0)

    # Carregar imagens de tratores (classe 1)
    for img_name in os.listdir(trator_dir):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(trator_dir, img_name)
            img = load_img(img_path, target_size=(150, 150))
            img_array = img_to_array(img) / 255.0
            imagens.append(img_array)
            rotulos.append(1)

    return np.array(imagens), np.array(rotulos)


## Carregamentos dos dados e divisão do treino

In [ ]:
# carregando imagens e rótulos
X, y = carregar_dataset(drone_dir, trator_dir)

# Divisão treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Converter rótulos para one-hot encoding
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)


## Aplicando data augmentation nos dados de treino

In [ ]:
# Gerador de alterações treino
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Gerador teste sem aplicação de alterações nas imagens
test_datagen = ImageDataGenerator()

# Preparar geradores de lotes para treino e teste
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size = 16
)

test_generator = test_datagen.flow(
    X_test, y_test,
    batch_size= 16
)


## Estruturando a CNN

In [ ]:
# Estrutura da CNN
model = Sequential([
    # 3 camdadas convolucionais
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape= (150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),

    # 1 camada densa
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    # Camada de saída
    Dense(2, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'melhor_modelo.h5',
    monitor='val_accuracy',
    save_best_only=True
)

## Treino do modelo

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 16,
    epochs=25,
    validation_data=test_generator,
    validation_steps=len(X_test) // 16,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/25
2/3 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - accuracy: 0.4822 - loss: 1.5254 

3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.5216 - loss: 1.3659 - val_accuracy: 0.4667 - val_loss: 0.6776
Epoch 2/25
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8750 - loss: 0.3244

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 724ms/step - accuracy: 0.8750 - loss: 0.3244 - val_accuracy: 0.5333 - val_loss: 0.6762
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.8984 - loss: 0.4629 - val_accuracy: 0.5333 - val_loss: 0.6767
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8889 - loss: 0.7582 - val_accuracy: 0.5333 - val_loss: 0.6760
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9167 - loss: 0.1497

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step - accuracy: 0.9219 - loss: 0.1444 - val_accuracy: 0.6000 - val_loss: 0.6702
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6667 - loss: 1.2277 - val_accuracy: 0.6000 - val_loss: 0.6628
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7674 - loss: 0.5604

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 462ms/step - accuracy: 0.7839 - loss: 0.5345 - val_accuracy: 0.7333 - val_loss: 0.6459
Epoch 8/25
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7778 - loss: 0.3968

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 407ms/step - accuracy: 0.7778 - loss: 0.3968 - val_accuracy: 0.8000 - val_loss: 0.6454
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.7269 - loss: 0.4438 - val_accuracy: 0.5333 - val_loss: 0.6517
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8125 - loss: 0.3752 - val_accuracy: 0.5333 - val_loss: 0.6528
Epoch 11/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8765 - loss: 0.3611 - val_accuracy: 0.6667 - val_loss: 0.6501
Epoch 12/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9375 - loss: 0.1234 - val_accuracy: 0.6667 - val_loss: 0.6499
Epoch 13/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.8712 - loss: 0.2778 - val_accuracy: 0.6667 - val_loss: 0.6348
Epoch 14/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.0811 - val_accuracy: 0.7333 - val_loss: 0.6314
Epoch 15/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.8828 - loss: 0.4277 - val_accuracy: 0.5333 - val_loss: 0.6341
Ep

## Acurácia do modelo

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step - accuracy: 0.7333 - loss: 0.6314
Acurácia no conjunto de teste: 73.33%


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Carregar o modelo treinado
model = load_model('melhor_modelo.h5')

# Pasta com as imagens para previsão
pasta_previsoes = '/content/drive/MyDrive/Entregavel_2/teste'

# Lista para armazenar as previsões
previsoes = []

# Iterar pelas imagens na pasta
for nome_arquivo in os.listdir(pasta_previsoes):
    if nome_arquivo.endswith(('.jpg', '.jpeg', '.png')):
        caminho_imagem = os.path.join(pasta_previsoes, nome_arquivo)
        try:
            img = load_img(caminho_imagem, target_size=(150, 150))
            img_array = img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            # Fazer a previsão
            predicao = model.predict(img_array)
            classe_predita = np.argmax(predicao)
            previsoes.append((nome_arquivo, classe_predita))

        except Exception as e:
            print(f"Erro ao processar a imagem {nome_arquivo}: {e}")

# Imprimir as previsões
for nome_arquivo, classe_predita in previsoes:
    print(f"Imagem: {nome_arquivo}, Classe Predita: {classe_predita}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Imagem: teste 1.jpg, Classe Predita: 1
Imagem: teste 2.jpg, Classe Predita: 0
Imagem: teste 3.jpg, Classe Predita: 1
Imagem: teste 4.jpeg, Classe Predita: 0
Imagem: teste 5.jpg, Classe Predita: 0
Imagem: teste 6.jpg, Classe Predita: 1
Imagem: teste 7.jpg, Classe Predita: 0
Imagem: teste 8.jpg, Classe Predita: 1


## Relatório

O código implementa uma Rede Neural Convolucional (CNN) para classificar
80 imagens de drones e tratores, sendo 40 imagens por objeto. A análise dos resultados obtidos será dividida em etapas para maior clareza:

1. Arquitetura da CNN:

A CNN possui 3 camadas convolucionais, intercaladas com camadas de Batch Normalization, Max Pooling e Dropout para regularização e evitar overfitting. A arquitetura termina com uma camada densa e uma camada de saída com função de ativação softmax para classificação binária.

2. Data Augmentation:

Foi utilizada técnica de Data Augmentation para aumentar a variabilidade dos dados de treino, incluindo rotações, translações e flips horizontais, tornando o modelo mais robusto.

3. Treinamento e Avaliação:

O modelo foi treinado com o otimizador Adam e função de perda 'categorical_crossentropy'. Métricas de precisão foram utilizadas para avaliar o desempenho. Early Stopping e Model Checkpoint foram implementados para evitar overfitting e salvar o melhor modelo.

4. Resultados:

A acurácia no conjunto de teste foi de aproximadamente 73,33%. É importante notar que o valor específico da acurácia pode variar a cada execução do código devido à aleatoriedade no processo de treinamento e Data Augmentation.

5. Análise:

A CNN apresentou resultados promissores na classificação de drones e tratores. A utilização de técnicas como Data Augmentation, Batch Normalization e Dropout contribuíram para a generalização do modelo e evitar overfitting. O uso de Early Stopping e Model Checkpoint permitiu encontrar um bom ponto de parada no treinamento e salvar o modelo com melhor desempenho.

7. Conclusão:

A CNN desenvolvida demonstra potencial para classificar imagens de drones e tratores. A análise dos resultados indica um bom desempenho, com acurácia satisfatória no conjunto de teste. No entanto, há espaço para melhorias através de ajustes na arquitetura, hiperparâmetros e técnicas de treinamento.


## Análise Comparativa: Modelo de Classificação (CNN) vs. Detectores de Objetos (YOLOv12 e YOLOv5)

A distinção crucial reside na tarefa que cada tipo de modelo foi projetado para realizar:

Modelo de Classificação (CNN): O objetivo principal de um modelo de classificação de imagens é atribuir um rótulo de classe a uma imagem inteira. Ele aprende a identificar as características visuais mais salientes que discriminam entre as categorias predefinidas. A métrica de avaliação primária é a acurácia, que mede a proporção de imagens corretamente classificadas. No nosso caso, a acurácia de 73.33% indica o desempenho do modelo em dizer "o que está na imagem". A perda (loss) complementa essa avaliação, quantificando o erro na atribuição das probabilidades das classes.

Modelos de Detecção de Objetos (YOLOv12 e YOLOv5): O objetivo desses modelos vai além da simples classificação. Eles visam identificar a presença de múltiplos objetos dentro de uma imagem, localizar cada objeto com uma caixa delimitadora e classificar cada objeto detectado. A saída não é apenas um rótulo para a imagem como um todo, mas sim múltiplos rótulos (um para cada objeto) juntamente com informações espaciais (as coordenadas das caixas delimitadoras). As métricas de avaliação primárias são a Mean Average Precision (mAP) em diferentes limiares de IoU (mAP50 e mAP50-95), que levam em conta tanto a precisão da classificação dos objetos detectados quanto a precisão da sua localização.

Comparação Direta e Implicações Técnicas:

Natureza da Saída: A saída de um classificador é uma única distribuição de probabilidade sobre as classes para a imagem de entrada. A saída de um detector de objetos é um conjunto de detecções, onde cada detecção inclui uma caixa delimitadora (definida por suas coordenadas), uma pontuação de confiança e uma probabilidade de classe. Essa diferença fundamental na natureza da saída reflete a complexidade inerentemente maior da tarefa de detecção.

Complexidade da Arquitetura e da Tarefa: As arquiteturas dos detectores de objetos, como YOLO, são geralmente mais complexas que as dos classificadores de imagem tradicionais. Elas precisam incorporar mecanismos para lidar com a variabilidade no número, tamanho, forma e localização dos objetos dentro de uma imagem. As camadas convolucionais servem como base para ambos, mas os detectores YOLO adicionam camadas especializadas para a previsão das caixas delimitadoras e a associação das predições às âncoras.

Métricas de Avaliação: A acurácia de um classificador não é diretamente comparável ao mAP de um detector de objetos. A acurácia avalia a correção da previsão da classe dominante na imagem, enquanto o mAP avalia a precisão e a revocação das detecções de múltiplos objetos, levando em conta a sobreposição espacial com as anotações verdadeiras. Um mAP alto indica que o detector é bom em encontrar todos os objetos relevantes (revocação) e que as detecções que ele faz são geralmente corretas (precisão), tanto em termos de classe quanto de localização.
Desempenho Relativo: Os resultados mostram que os modelos YOLO (v12 e v5) alcançam um mAP geral significativamente alto (acima de 90% em mAP50 para o melhor desempenho de cada modelo), indicando um bom desempenho na tarefa de detecção de tratores e drones. A acurácia do classificador (73.33%) reflete seu sucesso na tarefa de classificação, mas não informa sobre sua capacidade de localizar objetos individuais dentro da imagem ou de identificar múltiplos objetos.

Implicações Técnicas para os Resultados:

A alta acurácia do classificador sugere que as características distintivas das classes que ele foi treinado para reconhecer são bem aprendidas pela sua arquitetura CNN.

O alto mAP alcançado pelos modelos YOLO demonstra a eficácia das suas arquiteturas CNN especializadas em realizar a tarefa mais complexa de detecção de objetos, incluindo a localização precisa e a classificação simultânea. As diferenças no mAP entre YOLOv12 e YOLOv5 provavelmente se devem a variações nas suas arquiteturas CNN subjacentes, nas suas estratégias de previsão e nas suas funções de perda otimizadas para a detecção.

Conclusão da Comparação CNN (Classificação) vs. YOLO (Detecção):

Em essência, comparar a acurácia de um classificador com o mAP de um detector de objetos é como comparar maçãs com laranjas. Eles resolvem problemas fundamentalmente diferentes na visão computacional. Os resultados indicam que ambos os tipos de modelos foram capazes de aprender com sucesso as representações necessárias para suas respectivas tarefas. Os modelos YOLO, com suas arquiteturas CNN mais complexas e métricas de avaliação especializadas, demonstram a capacidade de não apenas classificar objetos, mas também de localizá-los com precisão dentro de uma imagem, uma capacidade que um classificador tradicional não possui. A escolha entre um classificador e um detector de objetos depende inteiramente do objetivo da aplicação: identificar o conteúdo geral de uma imagem ou identificar e localizar objetos específicos dentro dela.